<a href="https://colab.research.google.com/github/g95wang/twitter_keyword_extraction/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>